# Fitting a GLM with Gaussian Kernels

In [ ]:
# This cells setups the environment when executed in Google Colab.
try:
    import google.colab
    !curl -s https://raw.githubusercontent.com/ibs-lab/cedalion/dev/scripts/colab_setup.py -o colab_setup.py
    # Select branch with --branch "branch name" (default is "dev")
    %run colab_setup.py
except ImportError:
    pass

In [ ]:
import cedalion
import cedalion.sigproc.quality as quality
import cedalion.sigproc.motion_correct as motion_correct
import cedalion.sigproc.physio as physio


from cedalion import units
import cedalion.xrutils as xrutils
import cedalion.models.glm as glm
import cedalion.datasets

import cedalion.plots
from cedalion.plots import segmented_cmap

from pathlib import Path
import numpy as np
import xarray as xr

import matplotlib.pyplot as p
from statsmodels.stats.multitest import multipletests

xr.set_options(display_expand_data=False)
xrutils.unit_stripping_is_error()

## Load Data

In [ ]:
rec = cedalion.datasets.get_fingertappingDOT()

# assign better trial_type labels
rec.stim.cd.rename_events(
    {
        "1": "Control",
        "2": "FTapping/Left",
        "3": "FTapping/Right",
        "4": "BallSqueezing/Left",
        "5": "BallSqueezing/Right",
    }
)

In [ ]:
# count trials
rec.stim.groupby("trial_type")[["trial_type"]].count().rename(
    {"trial_type": "# trials"},
    axis=1,
)


## Trim dataset

Reduce the lenght of the dataset to about 5 minutes. This keeps computing times low for presentation and maintains 3 trials for each condition.
Also ignore the BallSqueezing conditions.

In [ ]:

tmin = 5
tmax = 315

rec.stim = rec.stim[
    (tmin <= rec.stim.onset)
    & (rec.stim.onset <= tmax)
    & (rec.stim.trial_type.isin(["Control", "FTapping/Left", "FTapping/Right"]))
]
rec["amp_cropped"] = rec["amp"].sel(time=slice(tmin,tmax))

# count trials
rec.stim.groupby("trial_type")[["trial_type"]].count().rename(
    {"trial_type": "# trials"},
    axis=1,
)


## Preprocessing 

- apply TDDR and wavelet corrections
- remove two known bad channels
- transform to concentrations
- apply a frequency filter

In [ ]:
rec["od"] = cedalion.nirs.int2od(rec["amp_cropped"])
rec["od_tddr"] = motion_correct.tddr(rec["od"])
rec["od_wavelet"] = motion_correct.wavelet(rec["od_tddr"])

bad_channels = ['S13D26', 'S14D28']

rec["od_clean"] = rec["od_wavelet"].sel(channel=~rec["od"].channel.isin(bad_channels))

# differential pathlength factors
dpf = xr.DataArray(
    [6, 6],
    dims="wavelength",
    coords={"wavelength": rec["amp"].wavelength},
)

rec["conc"] = cedalion.nirs.od2conc(rec["od_clean"], rec.geo3d, dpf, spectrum="prahl")

# Here we use a lowpass-filter to remove the cardiac component. 
# Drift will be modeled in the design matrix.

fmin = 0 * units.Hz
fmax = 0.5 * units.Hz

rec["conc_filtered"] = cedalion.sigproc.frequency.freq_filter(rec["conc"], fmin, fmax)

TS_NAME = "conc_filtered"

In [ ]:
display(rec[TS_NAME])

## Montage and Channel Distances

In [ ]:
cedalion.plots.plot_montage3D(rec["amp"], rec.geo3d)

In [ ]:
distances = cedalion.nirs.channel_distances(rec["amp"], rec.geo3d)

p.figure(figsize=(8,4))
p.hist(distances, 40)
p.axvline(22.5, c="r", ls="--")
p.xlabel("channel distance / mm")
p.ylabel("channel count");

The montage has longer (3-3.5cm) and shorter (~1.7-2.2cm) distance channels. Define a cut-off at 22.5 mm to define long and short channels.

In [ ]:
rec["ts_long"], rec["ts_short"] = cedalion.nirs.split_long_short_channels(
    rec[TS_NAME], rec.geo3d, distance_threshold=22.5 * units.mm
)

display(rec["ts_long"])
display(rec["ts_short"])

## Fitting a General Linear Model 

### Building the Design Matrix

Here we model the HRF with Gaussian kernels. For more modelling choices please refer to the [GLM basis functions notebook](./31_glm_basis_functions.ipynb).

In [ ]:

gaussian_kernels = glm.basis_functions.GaussianKernels(
    t_pre=2 * units.s,
    t_post=15 * units.s,
    t_delta=1.5 * units.s,
    t_std=2 * units.s,
)

hrf_basis = gaussian_kernels(rec["ts_long"])
for i in range(hrf_basis.sizes["component"]):
    p.plot(hrf_basis.time, hrf_basis[:,i])
    p.xlabel("relative time / s")
p.axvline(0, c="k")


Create design matrix from hrf, short channel and drift regressors.

When fitting a GLM to fNIRS data, there are regressors that apply to all channels. Other regressors differ between channel. For example, if one chooses for each channel the closest nearby short-channel to model superficial components, then this regressor differs between channels.

Consequently, the design matrix has two parts: common regressors and channel-wise regressors. 

In this example, all short channels are averaged to form a global (mostly) superficial component. The design matrix has only common regressors.

In [ ]:

dms = (
    glm.design_matrix.hrf_regressors(
        rec["ts_long"],
        rec.stim,
        gaussian_kernels,
    )
    & glm.design_matrix.drift_cosine_regressors(rec[TS_NAME], fmax=0.02 * units.Hz)
    & glm.design_matrix.average_short_channel_regressor(rec["ts_short"])
)

In [ ]:
display(dms)
display(dms.common)
display(dms.channel_wise)

### Visualize the design matrix

In [ ]:
# select common regressors
dm = dms.common
display(dm)

# using xr.DataArray.plot
f, ax = p.subplots(1,1,figsize=(12,10))
dm.sel(chromo="HbO", time=dm.time < 600).T.plot()
p.title("Shared Regressors")
#p.xticks(rotation=90)
p.show()

In [ ]:
# line plots of all regressors
f, ax = p.subplots(2,1,sharex=True, figsize=(12,6))

ch = "S5D6"

for i, chromo in enumerate(["HbO", "HbR"]):
    for reg in dm.regressor.values:
        label = reg if i == 0 else None
        ax[i].plot(dm.time, dm.sel(chromo=chromo, regressor=reg), label=label)

    for cwr in dms.channel_wise:
        for reg in cwr.regressor.values:
            label = reg if i == 0 else None
            ax[i].plot(cwr.time, cwr.sel(chromo=chromo, regressor=reg, channel=ch), label=label)

    cedalion.plots.plot_stim_markers(ax[i], rec.stim, y=1)
    ax[i].grid()
    ax[i].set_title(chromo)
    ax[i].set_ylim(-1.5,1.5)
f.suptitle("All Regressors for Channel " + ch)

f.legend(ncol=5, loc="upper center", bbox_to_anchor=(0.5, 0))
ax[0].set_xlim(0,240);
ax[1].set_xlabel("time / s");
f.set_tight_layout(True)

### Fitting the Model

 Choose between `noise_model='ols'` and `noise_model='ar_irls'`. 

In [ ]:
#results = glm.fit(rec["ts_long"], dms, noise_model="ols", max_jobs=-1)
results = glm.fit(rec["ts_long"], dms, noise_model="ar_irls", max_jobs=-1)

display(results)

### Inspecting Fit Results

The result of the fit is an array of statsmodels result objects. Each contains the fitted parameters and other functionality to inspect fit parameter uncertainty or to perform hypothesis tests.



In [ ]:
results[0,0].item().params

- separating drift / background regressors and signal
- plotting to show model-data fit

Cedalion provides the `.sm` accessor on arrays of result objects to make accessing the statsmodels functionality easier.

In [ ]:
display(results.sm.params)

The `glm.predict` function takes the original time series, the design matrix and the fitted parameters. Using these it predicts the sum of all regressors scaled with the best fitted parameters.

When only a subset of the fitted parameters is provided, only those regressors are considered. This allows to predict the signal or background components separately.

In [ ]:
predicted = glm.predict(rec["ts_long"], results.sm.params, dms)

predicted_background = glm.predict(
    rec["ts_long"],
    results.sm.params.sel(
        regressor=results.sm.params.regressor.str.startswith("Drift")
        | results.sm.params.regressor.str.startswith("short")
    ),
    dms,
)
predicted_signal = glm.predict(
    rec["ts_long"],
    results.sm.params.sel(regressor=results.sm.params.regressor.str.startswith("HRF")),
    dms,
)

meas_hrf_only = rec["ts_long"].pint.dequantify() - predicted_background 

Compare the preprocessed data to the full model

In [ ]:
channels = ["S3D1", "S12D22", "S9D25", "S11D31"]

f, ax = p.subplots(len(channels),1, figsize=(16, 3*len(channels)), sharex=True)

for i_ch, ch in enumerate(channels):
    ax[i_ch].plot(rec["ts_long"].time, rec["ts_long"].sel(channel=ch, chromo="HbO"), "-", c="orange", label="Data HbO")
    ax[i_ch].plot(rec["ts_long"].time, rec["ts_long"].sel(channel=ch, chromo="HbR"), "-", c="purple", label="Data HbR")
    ax[i_ch].plot(predicted_background.time, predicted.sel(channel=ch, chromo="HbO"), "r-", label="Model HbO")
    ax[i_ch].plot(predicted_background.time, predicted.sel(channel=ch, chromo="HbR"), "b-", label="Model HbR")
    #ax[i_ch].plot(predicted_background.time, predicted_background.sel(channel=ch, chromo="HbO"), "r-", alpha=.5, label="Model Drift HbO")
    #ax[i_ch].plot(predicted_background.time, predicted_background.sel(channel=ch, chromo="HbR"), "b-", alpha=.5, label="Model Drift HbO")
    cedalion.plots.plot_stim_markers(ax[i_ch], rec.stim, y=1)
    ax[i_ch].set_title(f"{ch} - {distances.sel(channel=ch).item().magnitude:.1f} mm")
    ax[i_ch].set_ylim(-1.75,1.75)
    ax[i_ch].legend(loc="upper left", ncol=8)

Subtract the components modelled by short and drift regressors and show the fitted activations

In [ ]:
f, ax = p.subplots(len(channels),1, figsize=(16, 3*len(channels)), sharex=True)

for i_ch, ch in enumerate(channels):
    ax[i_ch].plot(meas_hrf_only.time, meas_hrf_only.sel(channel=ch, chromo="HbO"), "-", c="orange", label="Data - Model Drift HbO" )
    ax[i_ch].plot(meas_hrf_only.time, meas_hrf_only.sel(channel=ch, chromo="HbR"), "-", c="purple", label="Data - Model Drift HbR")
    ax[i_ch].plot(predicted_signal.time, predicted_signal.sel(channel=ch, chromo="HbO"), "r-", label="Model HRF HbO")
    ax[i_ch].plot(predicted_signal.time, predicted_signal.sel(channel=ch, chromo="HbR"), "b-", label="Model HRF HbR")
    cedalion.plots.plot_stim_markers(ax[i_ch], rec.stim, y=1)

    ax[i_ch].legend(loc="upper left", ncol=8)
    ax[i_ch].set_title(ch)
    #ax[i_ch].set_ylim(-1.25, 1.25)
    ax[i_ch].set_ylim(-.5, .5)



### Plot goodness of fit metrics

Here the median absolute residuals (MAR) and $R^2$ are used.

(Note the scale difference between HbO and HbR for the MAR metric)

In [ ]:
mar = np.abs(predicted - rec["ts_long"].pint.dequantify()).median("time")

ss_res = ((rec["ts_long"].pint.dequantify() - predicted)**2).sum("time")
ss_tot = ((rec["ts_long"] - rec["ts_long"].mean("time"))**2).pint.dequantify().sum("time")
r2 = (1 - ss_res / ss_tot)

f,ax = p.subplots(2,2, figsize=(12,10))

for i, chromo in enumerate(["HbO", "HbR"]):
    cedalion.plots.scalp_plot(
        rec["ts_long"],
        rec.geo3d,
        # results.sm.pvalues.sel(regressor=reg, chromo=chromo),
        mar.sel(chromo=chromo),
        ax[0,i],
        cmap="RdYlGn_r",
        vmin=0,
        vmax={"HbO" : 0.15, "HbR" : 0.05}[chromo],
        title=f"MAR - {chromo}",
        cb_label="MAR / µM",
        channel_lw=2,
        optode_labels=True,
        optode_size=24,
    )
    cedalion.plots.scalp_plot(
        rec["ts_long"],
        rec.geo3d,
        r2.sel(chromo=chromo),
        ax[1,i],
        cmap="RdYlGn",
        vmin=0,
        vmax=1,
        title=f"$R^2$ - {chromo}",
        cb_label="$R^2$",
        channel_lw=2,
        optode_labels=True,
        optode_size=24,
    )

f.set_tight_layout(True)

### Hypothesis tests

- Performing a hypothesis test requires formulating null hypotheses.
- Here H0: the "size of activation" of left or right fingertapping does not differ from the control condition
- for simple models with one regressor per condition statsmodels offers and easy interface to formulate H0.

- how to express "size of activation" for a model with multiple gaussian kernels?

- Formulate H0 as a linear combination of the fitted parameters $\theta_i$ and a contrast vector $c_i$ and test if the result deviates from zero:

$$H0: x = \begin{pmatrix} \theta_1 \\ \theta_2 \\ \theta_3 \end{pmatrix} \cdot \begin{pmatrix} c_1 \\ c_2 \\ c_3 \end{pmatrix} = 0$$


$$ \textrm{for example: }c = \begin{pmatrix} 1 \\ 0 \\ -1 \end{pmatrix} \rightarrow H0: \theta_1 = \theta_3 $$

- Here we want to use the area under the HRF as size of the activation
- since the model is linear the area under the fitted HRF can be computed by scaling the areas of the gaussian kernels with the fitted parameters
$$A_{HRF} = \begin{pmatrix} \theta_1 \\ \theta_2 \\ \vdots \\ \theta_N \end{pmatrix} \cdot \begin{pmatrix} A_1 \\ A_2 \\ \vdots \\ A_n \end{pmatrix} $$

- **Use the area under each regressor as contrast with oppsite signs to formulate which conditions should be compared**

- restrict area calculation to a time window where the main activation is expected

In [ ]:
def gaussian_kernel_timewindowed_auc_contrast(
    dms, df_stim, condition1: str, condition2: str, tmin: float, tmax: float
):
    """This function computes contrast vectors based on the time-windowed are of the regressors."""

    time = dms.common.time

    # create two masks, that for each condition contains 1.0 only for 
    # time samples between onset+tmin and onset+tmax. All other entries
    # zero
    mask_cond1 = np.zeros(len(time))
    mask_cond2 = np.zeros(len(time))

    for _, row in df_stim.iterrows():
        t1, t2 = row["onset"]+tmin, row["onset"]+tmax
        if row["trial_type"].startswith(condition1):
            mask_cond1[(t1 <= time) & (time <= t2)] = 1.
        if row["trial_type"].startswith(condition2):
            mask_cond2[(t1 <= time) & (time <= t2)] = 1.

    # each gaussian regressor is multiplied with the mask for its condition. This sets
    # all parts of the regressor outside the time window to zero. Through integration the remaining 
    # area under the curve is calculated.

    nregressors = dms.common.sizes["regressor"] 
    contrast = np.zeros(nregressors)
    for i in range(nregressors):
        if dms.common.regressor.values[i].startswith(f"HRF {condition1}"):
            contrast[i] = np.trapezoid(dms.common[:,i,0]*mask_cond1, dms.common.time)
        if dms.common.regressor.values[i].startswith(f"HRF {condition2}"):
            contrast[i] = - np.trapezoid(dms.common[:,i,0]*mask_cond2, dms.common.time)

    return contrast

hypothesis_labels = [
    "HRF FTapping/Left = HRF Control",
    "HRF FTapping/Right = HRF Control",
]

hypotheses = [
    gaussian_kernel_timewindowed_auc_contrast(dms, rec.stim, "FTapping/Left", "Control", 5, 10),
    gaussian_kernel_timewindowed_auc_contrast(dms, rec.stim, "FTapping/Right", "Control", 5, 10),
]

display(hypotheses)

Using the calculated contrasts (i.e. hypotheses) perform a t-test in each channel and chromophore using `.sm`

In [ ]:
contrast_results = results.sm.t_test(hypotheses)

display(contrast_results)
display(contrast_results[0,0].item())

### Apply FDR-control and visualize corrected p-values for all channels

In [ ]:
# create a colormap for p-values
norm, cmap = cedalion.plots.segmented_cmap(
    "logpvalue",
    vmin=-4,
    vmax=0,
    segments=[
        (np.log10(0.0001), "blue"),
        (np.log10(0.001), "red"),
        (np.log10(0.005), "orange"),
        (np.log10(0.01), "yellow"),
        (np.log10(0.05), "green"),
        (np.log10(0.05), "lightgray"),
        (np.log10(1), "darkgray"),
    ],
    under="magenta"
)

nhypo = len(hypotheses)
f, ax = p.subplots(2,len(hypotheses),figsize=(6 * nhypo,8))

for i_row, chromo in enumerate(["HbO", "HbR"]):
    for i_hypo, hypo in enumerate(hypotheses):

        # get p_values for all channels and apply fdr correction
        # use Benjamini-Hochberg here
        p_values = contrast_results.sm.p_values().sel(chromo=chromo, hypothesis=i_hypo)
        _, q_values, _, _ = multipletests(p_values, alpha=0.05, method="fdr_bh")

        cedalion.plots.scalp_plot(
            rec["ts_long"],
            rec.geo3d,
            np.log10(q_values),
            ax[i_row][i_hypo],
            cmap=cmap,
            norm=norm,
            title=f"'{hypothesis_labels[i_hypo]}'?\n{chromo}",
            cb_label="BH-corrected p-value",
            channel_lw=2,
            optode_labels=False,
            cb_ticks_labels=[(np.log10(i), str(i)) for i in [0.0001, 0.001, 0.005, 0.01, 0.05, 1.]],
            optode_size=24,
        )


f.tight_layout()

## Extract and plot HRFs with uncertainties

- for extracting the HRF build a special design matrix
  - spans only the time of one trial
  - is not covolved over the stimulus duration (does not apply for gaussian kernels)

In [ ]:
dms_extract = glm.design_matrix.hrf_extract_regressors(rec["ts_long"], rec.stim, gaussian_kernels)
display(dms_extract.common)


Now predict time series with `dms_extract`. Additionally account for the uncertainties of the fitted parameters.

Using the covariance matrix of the fitted parameters and a multivariate
normal distribution, 10 sets of parameters are sampled around the best
fit value. From this ensemble of predictions the mean and std are returned.

In [ ]:
hrfs_control_mean, hrfs_control_std = glm.predict_with_uncertainty(
    rec["ts_long"],
    results,
    dms_extract,
    results.sm.params.regressor.str.startswith("HRF Control"),
)

for i_hypo, trial_type in enumerate(["HRF FTapping/Left", "HRF FTapping/Right"]):
    hrfs_mean, hrfs_std = glm.predict_with_uncertainty(
        rec["ts_long"],
        results,
        dms_extract,
        results.sm.params.regressor.str.startswith(trial_type),
    )

    p_values_hbo = contrast_results.sm.p_values().sel(chromo="HbO", hypothesis=i_hypo)
    p_values_hbr = contrast_results.sm.p_values().sel(chromo="HbR", hypothesis=i_hypo)

    # apply fdr correction (q-values are shown in captions)
    _, q_values_hbo, _, _ = multipletests(p_values_hbo, alpha=0.05, method="fdr_bh")
    _, q_values_hbr, _, _ = multipletests(p_values_hbr, alpha=0.05, method="fdr_bh")

    channels = hrfs_mean.channel.values

    f, ax = p.subplots(5, 9, figsize=(16, 9), sharex=True, sharey=True)
    ax = ax.flatten()

    for i_ch, ch in enumerate(channels[: len(ax)]):
        q_hbo = np.log10(q_values_hbo[i_ch])
        q_hbr = np.log10(q_values_hbr[i_ch])

        mm_hbo = hrfs_mean.sel(channel=ch, chromo="HbO")
        mm_hbr = hrfs_mean.sel(channel=ch, chromo="HbR")
        ss_hbo = hrfs_std.sel(channel=ch, chromo="HbO")
        ss_hbr = hrfs_std.sel(channel=ch, chromo="HbR")

        ax[i_ch].plot(mm_hbo.time, mm_hbo, "r-", label="HbO")
        ax[i_ch].fill_between(
            mm_hbo.time, mm_hbo - ss_hbo, mm_hbo + ss_hbo, fc="r", alpha=0.3
        )
        ax[i_ch].plot(mm_hbr.time, mm_hbr, "b-", label="HbR")
        ax[i_ch].fill_between(
            mm_hbr.time, mm_hbr - ss_hbr, mm_hbr + ss_hbr, fc="b", alpha=0.3
        )

        mm_hbo = hrfs_control_mean.sel(channel=ch, chromo="HbO")
        mm_hbr = hrfs_control_mean.sel(channel=ch, chromo="HbR")
        ss_hbo = hrfs_control_std.sel(channel=ch, chromo="HbO")
        ss_hbr = hrfs_control_std.sel(channel=ch, chromo="HbR")

        ax[i_ch].plot(mm_hbo.time, mm_hbo, "orange", label="HbO Control")
        ax[i_ch].fill_between(
            mm_hbo.time, mm_hbo - ss_hbo, mm_hbo + ss_hbo, fc="orange", alpha=0.3
        )
        ax[i_ch].plot(mm_hbr.time, mm_hbr, "magenta", label="HbR Control")
        ax[i_ch].fill_between(
            mm_hbr.time, mm_hbr - ss_hbr, mm_hbr + ss_hbr, fc="magenta", alpha=0.3
        )

        ax[i_ch].set_title(
            f"{ch}\nlogq: HbO:{q_hbo:.1f} HbR:{q_hbr:.1f}", fontdict={"fontsize": 8}
        )
        ax[i_ch].set_ylim(-0.3, 0.3)
        ax[i_ch].grid()
        ax[i_ch].axvline(0, c="k", lw=2)
        if i_ch == 0:
            ax[i_ch].legend(loc="upper left")
    f.suptitle(trial_type)
    f.set_tight_layout(True)
